In this notebook we look at the ability of `twinLab` to scale datasets.

First we import the necessary libraries: `numpy` allows us to efficiently manipulate arrays of numerical data; `pandas` gives us access to `DataFrames` which are a way of storing tabular data in `Python` and is the format used by `twinLab`. `matplotlib.pyplot` is used for plotting. `twinlab` is the machine-learning library we are using. Some of the libraries are renamed using `as` for convenience. 

In [ ]:
# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project imports
import twinlab as tl

In [ ]:
# Parameters
dataset_id = "data-scaling"
campaign_id = dataset_id
err_sig = 0.25
n_train = 100
n_eval = 101
random_seed = 42
n_cycle = 2

def f(x):
    return np.sin(2*np.pi*x*n_cycle)

In the cell below we generate our training dataset using the function and constants above. By uploading the dataset via tl.upload_dataset(), with the keyword argument `data-scaling`, the data will be scaled to values bewtween 0 and 1.

In [ ]:
# Seed the random-number generator
np.random.seed(random_seed)

# Training Data
X = np.random.rand(n_train)
y = f(X)+np.random.normal(0., err_sig, n_train)
df_train = pd.DataFrame({'X': X, 'y': y})
display(df_train)
tl.upload_dataset(df_train, dataset_id, verbose=True)

In [ ]:
# Evaluation data
eval = {"X": np.linspace(0, 1, n_eval)}
df_test = pd.DataFrame(eval)
display(df_test)

In [ ]:
# Training parameters
params = {
    "dataset_id": dataset_id,
    "inputs" : ["X"],
    "outputs": ["y"],
    "test_train_ratio": 1.,
}

# Plot parameters
grid = df_test["X"].values
alpha_fill = 0.25
ratios = [0.1, 0.2, 0.4, 0.8]
nrow, ncol = 2, 2
figx, figy = 4, 3

# Loop over different error in data and plot
plt.subplots(nrow, ncol, figsize=(ncol*figx, nrow*figy))
for iplot, ratio in enumerate(ratios):

    # Train model
    n = int(ratio*len(df_train))
    print("Number of data points used for training:", n)
    params["test_train_ratio"] = ratio
    tl.train_campaign(params, campaign_id, verbose=True)

    # Predict
    df_mean, df_std = tl.predict_campaign(df_test, campaign_id)
    mean, err = df_mean["y"].values, df_std["y"].values

    # Plot
    color = f"C{iplot}"
    plt.subplot(nrow, ncol, iplot+1)
    plt.plot(df_train["X"][:n], df_train["y"][:n], ".", color="black")
    plt.plot(grid, mean, "-", color=color, label=f"N = {n}")
    for nsig in [1, 2]:
        plt.fill_between(grid, mean-nsig*err, mean+nsig*err, lw=0, color=color, alpha=alpha_fill)
    # plt.xlabel("X"); plt.ylabel("y")
    plt.xticks([]); plt.yticks([])
    plt.ylim((-1.5, 1.5))
    plt.legend()

# Finalize plot
plt.tight_layout()
plt.show()

In [ ]:
# Delete campaign and dataset if necessary
tl.delete_campaign(campaign_id, verbose=True)
tl.delete_dataset(dataset_id, verbose=True)